In [1]:
import networkx as nx
import numpy as np
from pyquil import Program, get_qc
from pyquil.api import WavefunctionSimulator
from pyquil.gates import *
from pyquil.paulis import *

In [2]:
def apply_hadamards(L):
    '''
    Apply global Hadamard across all qubits contained in input list

    :param L list: list of qubits
    :return Program: pyQuil Program object applying the Hadamard gate to all input qubits
    '''
    p = Program()
    for q in L:
        p += Program(H(q))
    return p

In [3]:
def pauli_sum(L):
    '''
    Create a sum of Pauli-X operators over the qubits contained in the input list

    :param L list: list of qubits
    :return PauliSum: PauliSum of Pauli-X operators on the input qubits
    '''
    return sum([sX(q) for q in L])

In [4]:
def pauli_sum_maxcut(graph):
    '''
    Create a PauliSum corresponding to the MaxCut cost function

    :param graph Graph: NetworkX Graph object
    :return PauliSum: PauliSum object corresponding to MaxCut cost/objective function
    '''
    paulisum = 0
    # graph.adj is a dict with key: node, value: neighbors dict
    for i, nbrs in graph.adj.items():
        # nbrs is a dict itself with key: neighboring node, value: edge attributes dict
        # the weight of the edge can be extracted from eattr['weight']
        for j, eattr in nbrs.items():
            # add up contribution from current edge to paulisum
            w_ij = eattr['weight']
            Z_i = sZ(i)
            Z_j = sZ(j)
            paulisum += w_ij * 0.5 * (1 - Z_i * Z_j)
    paulisum *= 0.5
    return paulisum

In [5]:
def exponentiate_pauli_sum(paulisum, angle):
    '''
    Exponentiate a sum of Pauli operators to produce a unitary map

    :param paulisum PauliSum: PauliSum object containing the sum of Pauli operators
    :param angle float: specifying the coefficient of the paulisum in the exponential, with the convention
            e ^ (-i * angle * paulisum)
    :return Program: pyQuil program applying the exponentiated map
    '''
    p_exp = Program()
    for ps in paulisum:
        p_exp += exponential_map(ps)(angle)
    return p_exp

In [6]:
def run_qaoa(graph, beta, gamma):
    '''
    Run the QAOA prescription for the given graph and two angles

    :param graph Graph: NetworkX Graph object
    :param beta float: angle in [-pi, pi] range, appears in the mixer term
    :param gamma float: angle in [0, 2*pi] range, appears in the MaxCut cost term
    :return Program: pyQuil Program corresponding to the QAOA prescription in Eq (8) of problem sheet
    '''
    # check that values for angles are within valid ranges
    assert (beta >= -np.pi) and (beta <= np.pi), "beta is not within the range [-pi, pi]"
    assert (gamma >= 0) and (gamma <= 2*np.pi), "gamma is not within the range [0, 2*pi]"
    # create a list of qubits
    list_qubits = list(graph.nodes)
    # initialize program
    p = Program()
    # create equal-probability superposition as the starting state
    p += apply_hadamards(list_qubits)
    # apply U(gamma)
    paulisum_gamma = pauli_sum_maxcut(graph)
    p += exponentiate_pauli_sum(paulisum_gamma, gamma)
    # apply V(beta)
    paulisum_beta = pauli_sum(list_qubits)
    p += exponentiate_pauli_sum(paulisum_beta, beta)

    return p

In [7]:
def create_simple_two_node_graph():
    '''
    Simple two-node graph with edge weight = 1

    :return Graph: NetworkX Graph object with 2 nodes and edge weight=1
    '''
    G = nx.Graph()
    G.add_nodes_from(range(2))
    weighted_edges_list = [(0, 1, 1)]
    G.add_weighted_edges_from(weighted_edges_list)
    return G

In [8]:
def demo_optimize_angles():
    '''
    Demo program for optimizing angles for a simple 2-node graph

    :return tuple: optimal values for (beta, gamma)
    '''
    granularity = 2*np.pi/16.
    beta_range = np.arange(-np.pi, np.pi, granularity)
    gamma_range = np.arange(0, 2*np.pi, granularity)
    qvm = WavefunctionSimulator()
    graph = create_simple_two_node_graph()

    d_angles = {}
    for beta in beta_range:
        for gamma in gamma_range:
            prep_program = run_qaoa(graph, beta, gamma)
            hamiltonian = pauli_sum_maxcut(graph)
            expectation = np.array(qvm.expectation(prep_prog=prep_program, pauli_terms=hamiltonian))
            d_angles[(beta, gamma)] = expectation

    print ("Optimal values for (beta, gamma) for simple 2-node graph MAXCUT problem: ", max(d_angles, key=d_angles.get))

In [9]:
demo_optimize_angles()

/Users/tushar/anaconda3/lib/python3.6/site-packages/pyquil/paulis.py:665: UserWarning: The term Z1Z0 will be combined with Z0Z1, but they have different orders of operations. This doesn't matter for QVM or wavefunction simulation but may be important when running on an actual device.
  .format(t.id(sort_ops=False), first_term.id(sort_ops=False)))


Optimal values for (beta, gamma) for simple 2-node graph MAXCUT problem:  (-2.748893571891069, 1.5707963267948966)


In [10]:
# create wavefunction for optimal angles
beta, gamma = -2.748893571891069, 1.5707963267948966
graph = create_simple_two_node_graph()
prog = run_qaoa(graph, beta, gamma)
wfn_sim = WavefunctionSimulator()
wfn = wfn_sim.wavefunction(prog)
print (wfn)

-0.7071067812j|01> + -0.7071067812j|10>


/Users/tushar/anaconda3/lib/python3.6/site-packages/pyquil/paulis.py:665: UserWarning: The term Z1Z0 will be combined with Z0Z1, but they have different orders of operations. This doesn't matter for QVM or wavefunction simulation but may be important when running on an actual device.
  .format(t.id(sort_ops=False), first_term.id(sort_ops=False)))


In [12]:
# check that we have probability 1 of obtaining correct solution
# and probability 0 of obtaining incorrect solution
np.testing.assert_almost_equal(wfn.get_outcome_probs()['00'], 0.0)
np.testing.assert_almost_equal(wfn.get_outcome_probs()['01'], 0.5)
np.testing.assert_almost_equal(wfn.get_outcome_probs()['10'], 0.5)
np.testing.assert_almost_equal(wfn.get_outcome_probs()['11'], 0.0)